In [7]:
from utilities.utils_clean import *
from utilities.test_dataset_utils import *
from utilities.nlp_processing import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
test_data = load_from_pickle("../data/test_dataset_2015.pkl")

df_test_data = pd.DataFrame(test_data,index=["xml"],).T.reset_index()
df_test_data.columns = ["patentnumber","xml"] 


# df = read_xlsb_file()
# df[(df.issueyear == 2015) | (df.prophetic !=0)][["patentnumber","prophetic","nonprophetic","issueyear"]]
# df[df.issueyear == 2015].to_csv("freilichdataet_2015.csv")
df = pd.read_csv("../data/freilichdataet_2015.csv")
df["patentnumber"] = df["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))
df["patentnumber"] = df["patentnumber"].apply(remove_leadiong_zeros)
df.head()

Loaded 25081 patents from ../data/test_dataset_2015.pkl


,Unnamed: 0,patentnumber,appyear,issueyear,inventorcountry,divisional,continuation,prioritydate,priority.formatted,priorityyear,...,yr4,yr8,yr11,claims,wordaverage,litigated,orangebook,forward.cites,industry,subindustry
0,497653,8969641,1976.0,2015.0,US,0.0,0.0,19760903.0,28006.0,1976.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,19.0
1,497654,9134302,1995.0,2015.0,US,1.0,1.0,19830505.0,30441.0,1983.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
2,497655,8926966,1995.0,2015.0,US,1.0,1.0,19900921.0,33137.0,1990.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.0,31.0
3,497656,9181140,1993.0,2015.0,US,0.0,1.0,19920916.0,33863.0,1992.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN
4,497657,9199886,2009.0,2015.0,US,0.0,1.0,19940119.0,34353.0,1994.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [3]:
merged = df_test_data.merge(df, on="patentnumber", how="left")
print(len(merged))
cols = ["patentnumber","xml","prophetic","nonprophetic","allprophetic","someprophetic"]
merged[cols].head()

25081


,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
0,RE045323,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",1.0,12.0,0.0,1.0
1,RE045324,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
2,RE045325,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
3,8925349,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",26.0,0.0,1.0,0.0
4,8925551,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",6.0,5.0,0.0,1.0


In [8]:
def extract_examples_start_w_word_test(xml_siblings):
    examples = []
    current_example = None
    in_example = False

    for tag in xml_siblings:
        if tag.name == "heading":
            if (
                tag.text.strip().lower().startswith("example")
                or tag.text.strip().lower().startswith("experiment")
                or tag.text.strip().lower().startswith("test")
            ):
                in_example = True
                current_example = {
                    "number": tag.text.strip(),
                    "title": xml_siblings[xml_siblings.index(tag) + 1].text.strip(),
                    "content": [],
                }
                examples.append(current_example)
            # else:
            #     # If we hit any other heading, stop collecting content
            #     in_example = False
        elif in_example and tag.name == "p" and current_example is not None:
            current_example["content"].append(tag.text.strip())

    return examples

def dic_to_dic_w_tense_test(doc_w_exp,threshold = 50):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'


    for key, value in doc_w_exp.items():
        tense_counts = {"past": 0, "present": 0,"Unknown":0}
        
        if isinstance(value, list) and len(value) == 1 and len(value[0]["content"]) > threshold:
            desc = value[0]["title"] + "." + "".join(value[0]["content"])
            tense = check_tense_nltk(clean_text(desc))
            if tense !=  "Unknown":
                tense_counts[tense] += 1
            else:
                matches = re.findall(pattern, desc)
                if matches:
                    tense_counts["past"] += 1
                else:
                    tense_counts["Unknown"] += 1
            dic[key] = tense_counts


        elif isinstance(value, list) and len(value) > 1:
            for ls in value:
                if len(ls["content"]) > threshold:
                    desc = "".join(ls["content"])
                    tense = check_tense_nltk(clean_text(desc))
                    if tense !=  "Unknown":
                        tense_counts[tense] += 1
                    else:
                        matches = re.findall(pattern, desc)
                        if matches:
                            tense_counts["past"] += 1
                        else:
                            tense_counts["Unknown"] += 1
            dic[key] = tense_counts

        elif isinstance(value, dict):
            for ex, desc in value.items():
                if len(desc) > threshold:
                    tense = check_tense_nltk(clean_text(desc))
                    if tense !=  "Unknown":
                        tense_counts[tense] += 1
                    else:
                        matches = re.findall(pattern, desc)
                        if matches:
                            tense_counts["past"] += 1
                        else:
                            tense_counts["Unknown"] += 1
            dic[key] = tense_counts
        else:
            print(type(value))
            print(value)
            print(key)

    return dic

In [12]:
def extract_examples(merged, num_of_patents=1000):
    """
    Analyze patents to extract and compare example counts
    
    Args:
        merged: DataFrame containing patent data
        cols: Columns to use from the DataFrame
        num_of_patents: Number of patents to analyze (default=1000)
    
    Returns:
        dict: Statistics about the analysis results
    """
    mae = 0
    highest_difference = 0
    found_heading = 0
    not_found_heading = 0
    gib = 0
    short = 0
    mostdifss = []
    doc_w_exp = {}
    highest_difference_patent = None

    for row in merged[:num_of_patents].iterrows():
        xml = row[1]["xml"]
        if len(xml) <= 2000:
            short += 1
            continue
            
        s_tags = re.findall(r'<s\d+>.*?</s\d+>', xml)
        if len(s_tags) > 0:
            gib += 1
            continue
            
        heading = extract_experiments_w_heading(xml)
        janetsnumexamples = row[1]["prophetic"] + row[1]["nonprophetic"]

        # Process examples based on heading presence
        if heading:
            found_heading += 1
            examples = extract_examples_start_w_word_test(heading[0].find_next_siblings())
            if len(examples) == 0:
                soup = BeautifulSoup(xml, 'xml')
                siblings = soup.find_all(['heading', 'p'])
                examples = extract_examples_start_w_word_test(siblings)
        else:
            not_found_heading += 1
            soup = BeautifulSoup(xml, 'xml')
            siblings = soup.find_all(['heading', 'p'])
            examples = extract_examples_start_w_word_test(siblings)
            
        numexamples = len(examples)
        difference = abs(numexamples - janetsnumexamples)
        mae += difference
        
        if difference > 0:
            mostdifss.append([difference, row[1]["patentnumber"]])
        if difference > highest_difference:
            highest_difference = difference
            highest_difference_patent = row[1]["patentnumber"]
        if len(examples) > 0:
            doc_w_exp[row[1]["patentnumber"]] = examples

    results = {
        "found_heading": found_heading,
        "not_found_heading": not_found_heading,
        "gibberish": gib,
        "too_short": short,
        "mae": mae/num_of_patents,
        "total_error": mae,
        "highest_difference": highest_difference,
        "highest_difference_patent": highest_difference_patent,
        "most_differences": mostdifss,
        "documents_with_examples": doc_w_exp
    }
    
    return results


num_of_patents = 1000
stats = extract_examples(merged, num_of_patents=num_of_patents)
print(f"found_heading: {stats['found_heading']}, not_found_heading: {stats['not_found_heading']}, "
      f"gibberish: {stats['gibberish']}, too_short: {stats['too_short']}")
print(f"{num_of_patents} patents MAE: {stats['mae']}, total_error: {stats['total_error']}, "
      f"highest_difference: {stats['highest_difference']}, "
      f"highest_difference_patent: {stats['highest_difference_patent']}")

found_heading: 534, not_found_heading: 272, gibberish: 170, too_short: 24
1000 patents MAE: 0.33, total_error: 330.0, highest_difference: 56.0, highest_difference_patent: 8933072


In [14]:
import numpy as np
final = dic_to_dic_w_tense_test(stats["documents_with_examples"],threshold = 0)
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
print(f"number of Unknown tensed examples: {df_check['Unknown'].sum()}")
print(f"Total of examples in freilich {df_check.Total_Freilich.sum()},Total examples extracted {df_check.Total_Extracted.sum()}")
df_check.sort_values("Total_Mean_error",ascending=False).head(20)

d:\prophetic_patents\notebooks\utilities\nlp_processing.py:48: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


Number of exact matches: 386 out of 799 , Percentage: 48.310387984981226
Number of exact num of patent extracted: 681 out of 799
Avg Total error: 1.1739674593241551, num of corrects:  681
Avg Total prophetic error: 1.4217772215269087, num of corrects: 440
Avg Total nonprophetic error: 2.382978723404255, num of corrects: 401
3978.0
number of Unknown tensed examples: 670
Total of examples in freilich 11897.0,Total examples extracted 11359


,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
289,8927710,2,220,349,0.0,564.0,0.0,0.0,222,564.0,2.0,344.0,342.0,688.0
207,8927555,1,11,92,0.0,104.0,0.0,0.0,12,104.0,1.0,93.0,92.0,186.0
197,8927536,0,187,68,0.0,254.0,0.0,0.0,187,254.0,0.0,67.0,67.0,134.0
558,8933072,2,63,0,0.0,9.0,0.0,0.0,65,9.0,2.0,54.0,56.0,112.0
209,8927557,646,52,25,685.0,38.0,0.0,1.0,698,723.0,39.0,14.0,25.0,78.0
543,8933032,4,15,0,7.0,35.0,0.0,1.0,19,42.0,3.0,20.0,23.0,46.0
585,8933109,0,2,21,0.0,23.0,0.0,0.0,2,23.0,0.0,21.0,21.0,42.0
61,8926944,2,3,19,0.0,24.0,0.0,0.0,5,24.0,2.0,21.0,19.0,42.0
787,8937041,9,64,17,0.0,90.0,0.0,0.0,73,90.0,9.0,26.0,17.0,52.0
648,8933235,1,61,16,0.0,78.0,0.0,0.0,62,78.0,1.0,17.0,16.0,34.0
